In [1]:
import sys
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.transforms import *
glueContext = GlueContext(SparkContext.getOrCreate())

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1,application_1585141794068_0002,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

https://docs.aws.amazon.com/glue/latest/dg/aws-glue-api-crawler-pyspark-extensions-dynamic-frame.html

In [2]:
database = "userdata"
table_name = "userdata_json"

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
dyf  = glueContext.create_dynamic_frame.from_catalog(database=database, table_name=table_name)
print ("Count:  ", dyf.count())
dyf.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Count:   14902
root
|-- first: string
|-- last: string
|-- age: int
|-- gender: string
|-- latitude: double
|-- longitude: double

In [4]:
dyf.select_fields(['gender']).toDF().distinct().show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+
|gender|
+------+
|female|
|  male|
+------+

In [5]:
df = dyf.toDF()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
df.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+--------+---+------+--------+---------+
|  first|    last|age|gender|latitude|longitude|
+-------+--------+---+------+--------+---------+
|   tony|armstron| 45|  male|  7.6876|-138.0943|
|  kelly| griffin| 35|  male|-82.5088|  -8.7742|
|  milly| selstad| 33|female| 49.4328|  20.6618|
|  delia|  fleury| 32|female|-57.2241|   38.172|
|lucille| simmons| 41|female|-86.4193| 154.9268|
+-------+--------+---+------+--------+---------+
only showing top 5 rows

In [7]:
def map_gender(row):
    gender_map = {
        'male':1,
        'female':0
    }
    row['gender'] = gender_map[row['gender']]
    return row

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
dyf.map(map_gender).toDF().show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+--------+---+---------+------+-------+
|    last|latitude|age|longitude|gender|  first|
+--------+--------+---+---------+------+-------+
|armstron|  7.6876| 45|-138.0943|     1|   tony|
| griffin|-82.5088| 35|  -8.7742|     1|  kelly|
| selstad| 49.4328| 33|  20.6618|     0|  milly|
|  fleury|-57.2241| 32|   38.172|     0|  delia|
| simmons|-86.4193| 41| 154.9268|     0|lucille|
+--------+--------+---+---------+------+-------+
only showing top 5 rows

In [9]:
sdyf = dyf.toDF().repartition(1)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
sdyf.write.csv("s3://peter-data-bucket/sagemaker/output/userdata")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…